In [ ]:
!pip install -U accelerate
!pip install -U transformers
!pip install torch
!pip install evaluate
!pip install rouge
!pip install rouge_score
!pip install bert_score
!pip install gradio
!pip install typing_extensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
  Preparing meta

In [ ]:
from typing_extensions import Doc

In [ ]:
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers.optimization import AdamW
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import csv
from sklearn.model_selection import train_test_split

import gradio as gr
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np

In [ ]:
file_path='/content/sampled_qa_data.xlsx'

In [ ]:
df=pd.read_excel('/content/sampled_qa_data.xlsx')

In [ ]:
df.head(5)

,Q,A
0,"What is Q-Learning, and how does it work?",Q-Learning is a model-free Reinforcement Learn...
1,Are there ethical considerations regarding AI-...,"Yes, concerns about data privacy, transparency..."
2,Explain the workflow of building a machine lea...,The process typically involves importing datas...
3,Can logistic regression be used for classes mo...,No logistic regression cannot be used for clas...
4,When does regularization come into play in Mac...,At times when the model begins to underfit or ...


In [ ]:
# train_percentage = 0.8

# # Calculate the number of samples for training
# train_size = int(len(df) * train_percentage)

# # Split the dataset into training and testing sets
# df_train = df[:train_size]
# df_test = df[train_size:]
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
df.drop(df.columns[[0,3]], axis=1)

,Q,A
0,"What is Q-Learning, and how does it work?",Q-Learning is a model-free Reinforcement Learn...
1,Are there ethical considerations regarding AI-...,"Yes, concerns about data privacy, transparency..."
2,Explain the workflow of building a machine lea...,The process typically involves importing datas...
3,Can logistic regression be used for classes mo...,No logistic regression cannot be used for clas...
4,When does regularization come into play in Mac...,At times when the model begins to underfit or ...
...,...,...
662,What is the difference between BERT Base and B...,The difference between BERT Base and BERT Larg...
663,What is the difference between the Test set an...,The test set is used to test or evaluate the p...
664,How can cloud APIs facilitate collaborative ma...,Cloud APIs allow teams to easily share and dep...
665,Can you explain the architecture of FastAPI?,"FastAPI is built on top of Starlette, which is..."


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "gpt2"  #"gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained(checkpoint, pad_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# Load the GPT tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

# default to left padding
tokenizer.padding_side = "left"

In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
from torch.nn import functional as F

max_seq_length = 512

tokenized_data = []
for _, row in df_train.iterrows():
    question = row['Q']
    answer = row['A']
    question = " ".join(question.split()[:300])
    out = "<Question>\n" + str(question) + " \nAnswer\n " + str(answer) + " \n<ANSWER_ENDED>\n "
    out_ids = tokenizer(out,truncation = True, max_length = max_seq_length)


    tokenized_data.append(out_ids)



In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        return self.tokenized_data[idx]

# Create a custom dataset
custom_dataset = CustomTextDataset(tokenized_data)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='/content/gpt_model',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
)

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=custom_dataset,
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss
500,2.292100


TrainOutput(global_step=651, training_loss=2.167020548690116, metrics={'train_runtime': 118.4877, 'train_samples_per_second': 21.977, 'train_steps_per_second': 5.494, 'total_flos': 166681861632000.0, 'train_loss': 2.167020548690116, 'epoch': 3.0})

In [ ]:
model_output_path = "/content/model"

In [ ]:
 # Save the model
trainer.save_model(model_output_path)
# Save the tokenizer
tokenizer.save_pretrained(model_output_path)

('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.json',
 '/content/model/merges.txt',
 '/content/model/added_tokens.json')

In [ ]:
def generate_answer(question):
    input_ids = tokenizer.encode(question, return_tensors="pt").to("cuda")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids).to("cuda")
    pad_token_id = tokenizer.eos_token_id

    #output = model[0].generate(
    output = model.generate(
        input_ids,
        max_new_tokens=100,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    start_index = decoded_output.find("Answer")
    end_index = decoded_output.find("<ANSWER_ENDED>")

    if end_index != -1:
        # Extract the text between "Answer" and "<ANSWER_ENDED>"
        answer_text = decoded_output[start_index + len("Answer"):end_index].strip()
        return answer_text
    else:
        # If "<ANSWER_ENDED>" is not found, return the text following "Answer"
        answer_text = decoded_output[start_index + len("Answer"):].strip()
        return answer_text

    #return tokenizer.decode(output[0], skip_special_tokens=True)
    #return tokenizer.decode(output, skip_special_tokens=True)

In [ ]:
model_output_path='/content/drive/MyDrive/QA-model-280123'

In [ ]:
# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained(model_output_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
#model[0].to("cuda")
model.to("cuda")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
question="What is deep learning?"
response = generate_answer(question)
print("Generated Answer:", response)

Generated Answer: Deep learning (DL) is a subset of machine learning (ML) that uses neural networks to learn from data. It is a subset of machine learning (ML) that uses artificial neural networks to learn from data.


In [ ]:
df_test=pd.read_csv("/content/Q_test_data.csv")

In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="Generating answers")
df_test['generated_answers'] = df_test['Q'].progress_apply(lambda question: generate_answer(question))

Generating answers: 100%|██████████| 239/239 [04:30<00:00,  1.13s/it]


In [ ]:
df_test.head(5)

,Q,generated_answers
0,Can an SVM use gradient descent to maximize it...,Gradient descent can be used to maximize the m...
1,How to deal with the curse of dimensionality?,Dimensionality is a fundamental concept in com...
2,Importance of bias-variance tradeoff in ML?,Bias-variance tradeoff is a tradeoff between m...
3,"What is the main contribution of the paper ""Tr...",The main contribution of the paper is the deve...
4,Can KNN be used for regression tasks?,KNN is a widely used classification and regres...


In [ ]:
import re

def extract_answer(text):
    pattern = r'Answer(.*?)<ANSWER_ENDED>'
    result = re.search(pattern, text, re.DOTALL)

    if result:
        return result.group(1).strip()
    else:
        return text

In [ ]:
df_test['generated_answers'] = df_test['generated_answers'].apply(extract_answer)

In [ ]:
df_test.head(5)

,Q,A,generated_answers
668,What are some applications of deep learning?,Deep learning has numerous applications across...,Deep learning is used in various fields such a...
56,What is Kernel Trick in an SVM Algorithm?,Kernel Trick is a mathematical function which ...,Kernel trick is a technique in an SVM algorith...
851,What is the purpose of the text encoder in the...,The purpose of the text encoder is to take the...,The purpose of the text encoder in the Stable ...
895,What is the machine learning technique where a...,Transfer Learning,The machine learning technique involves traini...
722,How do LSTM units manage the flow of information?,"LSTM units have three gates - forget gate, inp...",LSTM units manage the flow of information by u...


In [ ]:
df_test.to_csv('model_Qtest_data_output.csv', index=False)

In [ ]:
df_test=pd.read_csv('/content/model_test_output.csv')

In [ ]:
df_test.columns

Index(['Unnamed: 0', 'Q', 'A', 'MQ', 'MA', 'generated_answers'], dtype='object')

In [ ]:
from evaluate import load
from collections import defaultdict
from tqdm import tqdm
from bert_score import score

def metric(data):
    Question,Answer,Generated_Answer  = ["Q","A","generated_answers"]
    rouge = load("rouge")
    meteor = load("meteor")
    score_bert = defaultdict(int)
    bertscore = load("bertscore")

    for _,x in tqdm(data.iterrows(),total=len(data)):

        rouge_ans = rouge.compute(predictions=[x[Generated_Answer]],references=[x[Answer]])

        score_bert["rouge_1_ans"] += rouge_ans['rouge1']
        score_bert["rouge_2_ans"] += rouge_ans['rouge2']
        score_bert["rouge_L_ans"] += rouge_ans['rougeL']

        meteor_ans = meteor.compute(predictions=[x[Generated_Answer]],references=[x[Answer]])

        score_bert["meteor_ans"] += meteor_ans["meteor"]

        # BERT score
        results = bertscore.compute(predictions=[x[Generated_Answer]], references=[x[Answer]], lang="en")

        score_bert["bert_score_f1"] += results["f1"][0]

    print("The evalution scores are as follows:")

    for name,val in score_bert.items():
        print(f"{name}: {val/len(data)}")




In [ ]:
metric(df_test)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


  0%|          | 0/96 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 96/96 [00:43<00:00,  2.21it/s]

The evalution scores are as follows:
rouge_1_ans: 0.2798084047494616
rouge_2_ans: 0.10548904502165342
rouge_L_ans: 0.22985981659946578
meteor_ans: 0.24160376248581353
bert_score_f1: 0.8614757383863131


In [ ]:
!transformers-cli login

2024-01-20 05:33:17.894451: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-20 05:33:17.894500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-20 05:33:17.895716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-20 05:33:19.048781: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ERROR! `huggingface-cli login` uses an outdated login mechanism that is not compatible with the Hugging Face Hub backend anymore. Please use `huggingface-cli login instead.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
my_model.push_to_hub("qa-part2"),

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

(CommitInfo(commit_url='https://huggingface.co/sandhyac/qa-part2/commit/9a3d5efdaecccc347d278c4956bcc3fd155b817d', commit_message='Upload model', commit_description='', oid='9a3d5efdaecccc347d278c4956bcc3fd155b817d', pr_url=None, pr_revision=None, pr_num=None),)

In [ ]:
from transformers import GPT2Model, GPT2Config, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sandhyac/qa-part2"),

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
tokenizer.push_to_hub("qa-part2"),

(CommitInfo(commit_url='https://huggingface.co/sandhyac/qa-part2/commit/465f69e5b666a3350b70ed0e7eb398e28e69e714', commit_message='Upload tokenizer', commit_description='', oid='465f69e5b666a3350b70ed0e7eb398e28e69e714', pr_url=None, pr_revision=None, pr_num=None),)

In [ ]:
!huggingface-cli repo create qa-part2,

git version 2.34.1
git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)

You are about to create sandhyac/qa-part2,
Proceed? [Y/n] Y
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 49, in main
    service.run()
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/user.py", line 174, in run
    url = self._api.create_repo(
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py", line 110, in _inner_fn
    validate_repo_id(arg_value)
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py", line 164, in validate_repo_id
    raise HFValidationError(
huggingface_hub.utils._validators.HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'sandhyac/

In [ ]:
# Create Gradio interface with a text input and a submit button

text_input = gr.Textbox()

iface = gr.Interface(
    fn=generate_answer,
    inputs=text_input,
    outputs='text',
    live=False,  # Set live to False to use the button
)
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a158045e221ecca21c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
